In [1]:
import pandas as pd
import numpy as np

# Load abundance data
nafld_abd = pd.read_csv('../data/NAFLD/abd.tsv', sep='\t', index_col=0)
metadata = pd.read_csv('../data/NAFLD/NASH_forward_63_map.txt', sep='\t')
metadata.columns = [col.replace('#', '') for col in metadata.columns]
nash_samples = metadata[metadata['DiseaseStatus'] == 'NASH']['SampleID'].tolist()
nash_abundance = nafld_abd.loc[:, nafld_abd.columns.isin(nash_samples)]

# Filter tax
threshold = 0.8
sample_count = nash_abundance.shape[1]
min_samples = int(sample_count * threshold)
present_in_most = (nash_abundance > 0).sum(axis=1) >= min_samples
nash_abundance_filtered = nash_abundance.loc[present_in_most]
nash_abundance_filtered.index = nash_abundance_filtered.index.str.replace('s__', '')
output_filename = f'nash_abundance_filtered_threshold_{int(threshold*100)}.tsv'
nash_abundance_filtered.to_csv(output_filename, sep='\t')

# Print summary
print(f"Total samples: {len(metadata)}")
print(f"NASH samples: {len(nash_samples)}")
print(f"Original species: {len(nash_abundance.index)}")
print(f"Filtered species: {len(nash_abundance_filtered.index)}")
print(f"Species present in at least {threshold*100}% of samples: {len(nash_abundance_filtered.index)}")

# Calculate filtered percentage
filtered_ratio = (len(nash_abundance.index) - len(nash_abundance_filtered.index)) / len(nash_abundance.index) * 100
print(f"Percentage of species filtered: {filtered_ratio:.2f}%")

Total samples: 63
NASH samples: 22
Original species: 97
Filtered species: 33
Species present in at least 80.0% of samples: 33
Percentage of species filtered: 65.98%


## Make causal inference network [optional]
Those steps follow the NAFLD cohort(10.1002/imt2.61) provided scripts https://github.com/tjcadd2020/NAFLD_keystone/blob/master/NAFLD_Discovery_20190916_Final.ipynb to get the causal network.


### 1. SparCC
We recommand install in a new conda enviroment for python confilct.

```conda create --name sparcc python=2.7```

```conda active sparcc```

```pip install numpy==1.10.1```

```pip install pandas```

Download yonatanf-sparcc-3aff6141c3f1.zip from https://github.com/tjcadd2020/NAFLD_keystone/tree/master/pkg and unzip
Make sure the scipt is in ```yonatanf-sparcc-3aff6141c3f1/``` not ```yonatanf-sparcc-3aff6141c3f1/yonatanf-sparcc-3aff6141c3f1/```

```python run_sparcc.py```

input
- nash_abundance_filtered_threshold_80.tsv
output
- NASH_F_corr_threshold_80.tsv
- NASH_F_p_threshold_80_one_sided.tsv

### 2.Causal inference
```conda create --name dowhy python=3.6```

```conda active dowhy```

```conda install conda-forge::dowhy```

```python run_causalinference.py```
output
- cancer_causal_threshold_80/
- threshold_80_NASH_F_causal_p_one_sided.tsv
- threshold_80_NASH_F_causal.tsv


In [2]:
# Load causal inference

A = pd.read_csv('cancer_causal_threshold_80/cancer_ematrix_2.csv', index_col=0, sep=',')
prior = pd.read_csv('cancer_causal_threshold_80/cancer_pmatrix_2.csv', index_col=0, sep=',')
A *= (prior>=0.9)

In [3]:
import numpy as np
import pandas as pd
import random

def prepare_sample_for_perturbation(abundance_file, sample_idx=0):
    """
    Prepare a single sample's relative abundance for perturbation analysis
    
    Args:
        abundance_file: str, TSV file path
        sample_idx: int, index of sample to select (default 0)
    
    Returns:
        sample: Series, processed sample relative abundance
    """

    abundance_df = pd.read_csv(abundance_file, sep='\t', index_col=0)
    
    sample = abundance_df.iloc[:, sample_idx]
    
    if np.abs(sample.sum() - 1.0) > 0.01: 
        sample = sample / sample.sum()
    
    sample = sample.clip(lower=0)
    
    return sample

def get_delta_X(A, inventions, abundances, decay=0.5, max_iter=100):
    """Calculate abundance changes after network propagation"""
    for it in range(max_iter):
        inventions += np.round(np.dot(A, inventions) * abundances * decay**(it), 5)
    return inventions

def allocate_reduction(species_list, total_reduction, sample, OTUs):
    """
    Allocate reduction amount to selected species
    
    Args:
        species_list: list, species to reduce
        total_reduction: float, total amount to reduce
        sample: Series, original abundances
        OTUs: list, all OTUs
    
    Returns:
        inventions: array, intervention vector
    """
    inventions = np.zeros(len(OTUs))
    n_species = len(species_list)
    
    if n_species == 1:
        idx = OTUs.index(species_list[0])
        inventions[idx] = -total_reduction
    else:
        split_points = np.random.uniform(0, total_reduction, n_species-1)
        split_points = np.sort(split_points)
        
        reductions = np.diff(np.concatenate(([0], split_points, [total_reduction])))
        
        for otu, red in zip(species_list, reductions):
            idx = OTUs.index(otu)
            inventions[idx] = -red
            
    return inventions

def simulate_relative_abundance_perturbation(sample, A_matrix, total_reduction=0.05, 
                                           primary_candidate_otus=None, secondary_candidate_otus=None,
                                           decay=0.5, max_iter=100, random_seed=42):
    """
    Simulate random perturbation of relative abundance
    
    Uses primary candidates first, then secondary candidates if needed
    """
    random.seed(random_seed)
    np.random.seed(random_seed)
    
    OTUs = list(A_matrix.columns)
    original_sum = sample.sum()
    reduction_amount = original_sum * total_reduction
    
    primary_valid = []
    secondary_valid = []
    
    if primary_candidate_otus is None and secondary_candidate_otus is None:
        candidate_otus = OTUs
        n_species_to_reduce = max(1, int(len(candidate_otus) * np.random.uniform(0.2, 0.8)))
        selection_probs = sample / sample.sum()
        try:
            perturb_otus = list(np.random.choice(
                candidate_otus,
                size=min(n_species_to_reduce, len(candidate_otus)),
                replace=False,
                p=selection_probs
            ))
            inventions = allocate_reduction(perturb_otus, reduction_amount, sample, OTUs)
        except ValueError:
            perturb_otus = [sample.idxmax()]
            inventions = np.zeros(len(OTUs))
            inventions[OTUs.index(perturb_otus[0])] = -reduction_amount
    else:
        if primary_candidate_otus is not None:
            primary_valid = [otu for otu in primary_candidate_otus if otu in OTUs]
        
        if secondary_candidate_otus is not None:
            secondary_valid = [otu for otu in secondary_candidate_otus if otu in OTUs]
        
        if primary_valid:
            primary_sample = sample[primary_valid]
            primary_total = primary_sample.sum()
            
            if primary_total >= reduction_amount:
                n_primary_to_reduce = max(1, int(len(primary_valid) * np.random.uniform(0.2, 0.8)))
                selection_probs = primary_sample / primary_sample.sum()
                try:
                    perturb_otus = list(np.random.choice(
                        primary_valid,
                        size=min(n_primary_to_reduce, len(primary_valid)),
                        replace=False,
                        p=selection_probs
                    ))
                    inventions = allocate_reduction(perturb_otus, reduction_amount, sample, OTUs)
                except ValueError:
                    perturb_otus = [primary_sample.idxmax()]
                    inventions = np.zeros(len(OTUs))
                    inventions[OTUs.index(perturb_otus[0])] = -reduction_amount
            else:
                perturb_otus = primary_valid.copy()
                inventions = np.zeros(len(OTUs))
                
                for otu in primary_valid:
                    idx = OTUs.index(otu)
                    inventions[idx] = -sample[otu]
                
                remaining_reduction = reduction_amount - primary_total
                
                if secondary_valid:
                    secondary_sample = sample[secondary_valid]
                    n_secondary_to_reduce = max(1, int(len(secondary_valid) * np.random.uniform(0.2, 0.8)))
                    selection_probs = secondary_sample / secondary_sample.sum()
                    
                    try:
                        additional_otus = list(np.random.choice(
                            secondary_valid,
                            size=min(n_secondary_to_reduce, len(secondary_valid)),
                            replace=False,
                            p=selection_probs
                        ))
                        perturb_otus.extend(additional_otus)
                        
                        secondary_inventions = allocate_reduction(additional_otus, remaining_reduction, sample, OTUs)
                        inventions += secondary_inventions
                        
                    except ValueError:
                        max_otu = secondary_sample.idxmax()
                        perturb_otus.append(max_otu)
                        inventions[OTUs.index(max_otu)] = -remaining_reduction
        
        elif secondary_valid: 
            secondary_sample = sample[secondary_valid]
            n_secondary_to_reduce = max(1, int(len(secondary_valid) * np.random.uniform(0.2, 0.8)))
            selection_probs = secondary_sample / secondary_sample.sum()
            
            try:
                perturb_otus = list(np.random.choice(
                    secondary_valid,
                    size=min(n_secondary_to_reduce, len(secondary_valid)),
                    replace=False,
                    p=selection_probs
                ))
                inventions = allocate_reduction(perturb_otus, reduction_amount, sample, OTUs)
            except ValueError:
                perturb_otus = [secondary_sample.idxmax()]
                inventions = np.zeros(len(OTUs))
                inventions[OTUs.index(perturb_otus[0])] = -reduction_amount
                
    # Calculate network propagation effects
    delta_abundance = get_delta_X(A_matrix, inventions, sample, decay, max_iter)
    
    # Calculate final abundance
    final_abundance = sample + delta_abundance
    
    # Ensure non-negative abundance
    final_abundance = np.maximum(final_abundance, 0)
    return pd.Series(final_abundance, index=OTUs), perturb_otus, inventions

def calculate_FR(abundances, distance_file="../data/NAFLD/NASH_distance.tsv"):
    """
    Calculate Functional Redundancy (FR)
    FR = sum(dij * pi * pj) where i ≠ j
    
    Args:
        abundances: Series, species abundances
        distance_file: str, path to species distance matrix
    """
    distances = pd.read_csv(distance_file, sep='\t', index_col=0)
    
    distances.index = distances.index.str.replace('s__', '')
    distances.columns = distances.columns.str.replace('s__', '')
    
    common_species = sorted(set(abundances.index) & set(distances.index))
    
    if not common_species:
        raise ValueError("No common species found between abundance data and distance matrix")
    
    abundances = abundances[common_species]
    distances = distances.loc[common_species, common_species]
    
    fr = 0
    n = len(common_species)
    
    for i in range(n):
        for j in range(n):
            if i != j:
                species_i = common_species[i]
                species_j = common_species[j]
                fr += distances.iloc[i, j] * abundances[species_i] * abundances[species_j]
    
    return fr

In [4]:
sample = prepare_sample_for_perturbation('nash_abundance_filtered_threshold_80.tsv')
keystone_species = ['B. caecicola-OTU4']
keystone_cluster = ['B. caecicola-OTU4','F. prausnitzii-OTU80','B. producta-OTU32','F. prausnitzii-OTU8','Ezakiella-OTU94','A. ruminis-OTU75','C. pinnipediorum','B. coprophilus','Escherichia-Shigella-OTU88','F. magna','P. buccalis','A. octavius','B. barnesiae','P. sp. 2007b','P. ivorii','D. pneumosintes','P. loveana','P. olsenii','E. peruensis','P. lacrimalis','S5-A14a-OTU76']
other_species = [sp for sp in sample.index if sp not in keystone_cluster]

In [5]:
# Load file to get sample count
df = pd.read_csv('nash_abundance_filtered_threshold_80.tsv', sep='\t')
n_samples = df.shape[1] 

# Set random seed
seed = 42
np.random.seed(seed)

# 生成100次模拟的参数
n_simulations = 100
simulation_params = pd.DataFrame({
    'simulation_id': range(n_simulations),
    'sample_idx': np.random.randint(0, n_samples-1, size=n_simulations), 
    'random_seed': np.random.randint(0, 1000, size=n_simulations)
})

# Save parameters to file
output_filename = f'simulation_params_root_seed_{seed}.tsv'
simulation_params.to_csv(output_filename, sep='\t', index=False)

print("Simulation parameters preview:")
print(simulation_params.head(10))
print("\nParameter statistics:")
print(f"Total sample count: {n_samples}")
print(f"Sample index range: {simulation_params['sample_idx'].min()}-{simulation_params['sample_idx'].max()}")
print(f"Random seed range: {simulation_params['random_seed'].min()}-{simulation_params['random_seed'].max()}")
print(f"\nParameters saved to file: {output_filename}")

Simulation parameters preview:
   simulation_id  sample_idx  random_seed
0              0           6          378
1              1          19          772
2              2          14          489
3              3          10          230
4              4           7           40
5              5          20           27
6              6           6          134
7              7          18          200
8              8          10          839
9              9          10          779

Parameter statistics:
Total sample count: 23
Sample index range: 0-21
Random seed range: 4-986

Parameters saved to file: simulation_params_root_seed_42.tsv


In [6]:
def run_simulation_with_reduction(seed, total_reduction):
    # Load simulation parameters
    params_df = pd.read_csv(f'simulation_params_root_seed_{seed}.tsv', sep='\t')
    params_df['random_seed'] = params_df['random_seed'].astype(int)

    columns = ['simulation_id', 'sample_idx', 'random_seed', 
              'total_reduction',  # 新增total_reduction列
              'original_FR',
              'keystone_species_FR', 'keystone_cluster_FR', 
              'random_FR', 'other_species_FR']
    results_df = pd.DataFrame(columns=columns)

    # Run 100 simulations
    for _, row in params_df.iterrows():
        sim_id = int(row['simulation_id'])
        sample_idx = int(row['sample_idx'])
        sim_seed = int(row['random_seed'])
        
        print(f"Running simulation {sim_id}, sample_idx={sample_idx}, seed={sim_seed}, reduction={total_reduction}")
        
        sample = prepare_sample_for_perturbation('nash_abundance_filtered_threshold_80.tsv', 
                                               sample_idx=sample_idx)
        
        original_FR = calculate_FR(sample)
        
         # 1. Keystone species perturbation
        result_keystone = simulate_relative_abundance_perturbation(
            sample, A,
            primary_candidate_otus=keystone_species,
            secondary_candidate_otus=keystone_cluster,
            total_reduction=total_reduction,  
            random_seed=sim_seed
        )
        keystone_FR = calculate_FR(result_keystone[0])
        
        # 2. Keystone cluster perturbation
        result_cluster = simulate_relative_abundance_perturbation(
            sample, A,
            primary_candidate_otus=keystone_cluster,
            secondary_candidate_otus=other_species,
            total_reduction=total_reduction,  
            random_seed=sim_seed
        )
        cluster_FR = calculate_FR(result_cluster[0])
        
        # 3. Random perturbation
        result_random = simulate_relative_abundance_perturbation(
            sample, A,
            total_reduction=total_reduction, 
            random_seed=sim_seed
        )
        random_FR = calculate_FR(result_random[0])
        
        # 4. Other species perturbation
        result_other = simulate_relative_abundance_perturbation(
            sample, A,
            primary_candidate_otus=other_species,
            secondary_candidate_otus=keystone_cluster,
            total_reduction=total_reduction,  
            random_seed=sim_seed
        )
        other_FR = calculate_FR(result_other[0])
        
        
        results_df.loc[len(results_df)] = [
            sim_id, sample_idx, sim_seed,
            total_reduction, 
            original_FR,
            keystone_FR, cluster_FR,
            random_FR, other_FR
        ]
        
        if (sim_id + 1) % 10 == 0:
            print(f"Completed {sim_id + 1} simulations")

    # Save results
    output_filename = f'simulation_results_root_seed_{seed}_reduction_{total_reduction}.tsv'
    results_df.to_csv(output_filename, sep='\t', index=False)

    print(f"\nAll simulations completed, results saved to: {output_filename}")
    print("\nResults preview:")
    print(results_df.head())

    print("\nBasic statistics:")
    for col in results_df.columns[4:]: 
        print(f"\n{col}:")
        print(results_df[col].describe())
    
    return results_df

seed = 42

# Different reduction values
reductions = [0.05, 0.1, 0.15, 0.2] 
all_results = {}

for reduction in reductions:
    print(f"\nRunning simulations with total_reduction = {reduction}")
    all_results[reduction] = run_simulation_with_reduction(seed, reduction)


Running simulations with total_reduction = 0.05
Running simulation 0, sample_idx=6, seed=378, reduction=0.05
Running simulation 1, sample_idx=19, seed=772, reduction=0.05
Running simulation 2, sample_idx=14, seed=489, reduction=0.05
Running simulation 3, sample_idx=10, seed=230, reduction=0.05
Running simulation 4, sample_idx=7, seed=40, reduction=0.05
Running simulation 5, sample_idx=20, seed=27, reduction=0.05
Running simulation 6, sample_idx=6, seed=134, reduction=0.05
Running simulation 7, sample_idx=18, seed=200, reduction=0.05
Running simulation 8, sample_idx=10, seed=839, reduction=0.05
Running simulation 9, sample_idx=10, seed=779, reduction=0.05
Completed 10 simulations
Running simulation 10, sample_idx=20, seed=929, reduction=0.05
Running simulation 11, sample_idx=3, seed=32, reduction=0.05
Running simulation 12, sample_idx=7, seed=47, reduction=0.05
Running simulation 13, sample_idx=2, seed=502, reduction=0.05
Running simulation 14, sample_idx=21, seed=406, reduction=0.05
R

In [7]:
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd

# Load data for different reduction values
seed = 42
reductions = [0.05, 0.1, 0.15, 0.2]

all_results = []

for reduction in reductions:
    df = pd.read_csv(f'simulation_results_root_seed_{seed}_reduction_{reduction}.tsv', sep='\t')
    df['reduction'] = reduction  # Add reduction column
    all_results.append(df)

# Combine all data
combined_results = pd.concat(all_results)

# Prepare data for plotting
fr_columns = ['keystone_species_FR', 'keystone_cluster_FR', 'random_FR', 'other_species_FR']
plot_data = combined_results[fr_columns + ['reduction']].melt(id_vars=['reduction'])

# Set figure size
plt.figure(figsize=(12, 6))

# Create boxplot
sns.boxplot(x='reduction', y='value', hue='variable', data=plot_data)

# Improve labels
plt.xlabel('Reduction')
plt.ylabel('FR Value')
plt.title('FR Values Across Different Reduction Levels')

# Update legend labels
new_labels = ['Keystone Species', 'Keystone Cluster', 'Random', 'Other Species']
handles, _ = plt.gca().get_legend_handles_labels()
plt.legend(handles, new_labels, title='Perturbation Type', bbox_to_anchor=(1.05, 1), loc='upper left')

# Save figure
plt.tight_layout()
plt.savefig(f'FR_boxplot_root_seed_{seed}_all_reductions.png', bbox_inches='tight')
plt.close()

# Statistical analysis for each reduction value
for reduction in reductions:
    results_df = combined_results[combined_results['reduction'] == reduction]
    
    print(f"\n=== Statistical Analysis (Reduction = {reduction}) ===")
    
    # Perform pairwise Mann-Whitney U tests
    perturbation_pairs = []
    for i in range(len(fr_columns)):
        for j in range(i+1, len(fr_columns)):
            stat, p_value = stats.mannwhitneyu(
                results_df[fr_columns[i]], 
                results_df[fr_columns[j]],
                alternative='two-sided'
            )
            perturbation_pairs.append({
                'comparison': f'{fr_columns[i]} vs {fr_columns[j]}',
                'statistic': stat,
                'p_value': p_value
            })
    
    # Create DataFrame for test results
    stats_df = pd.DataFrame(perturbation_pairs)
    
    print(f"\nMann-Whitney U test results:")
    print(stats_df)
    
    print(f"\nBasic statistics:")
    print(results_df[fr_columns].describe())


=== Statistical Analysis (Reduction = 0.05) ===

Mann-Whitney U test results:
                                   comparison  statistic   p_value
0  keystone_species_FR vs keystone_cluster_FR     4429.0  0.163245
1            keystone_species_FR vs random_FR     4290.0  0.082934
2     keystone_species_FR vs other_species_FR     3825.0  0.004101
3            keystone_cluster_FR vs random_FR     4726.0  0.503962
4     keystone_cluster_FR vs other_species_FR     4287.0  0.081698
5               random_FR vs other_species_FR     4494.0  0.216781

Basic statistics:
       keystone_species_FR  keystone_cluster_FR   random_FR  other_species_FR
count           100.000000           100.000000  100.000000        100.000000
mean              0.385652             0.395820    0.400797          0.412171
std               0.077267             0.076525    0.077546          0.076237
min               0.218626             0.206447    0.210246          0.225651
25%               0.343569             0.35

In [8]:
import os
import shutil
import glob

output_dir = '../result/perturbation_simulation/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Move simulation result files
shutil.move("nash_abundance_filtered_threshold_80.tsv", os.path.join(output_dir,"nash_abundance_filtered_threshold_80.tsv"))
shutil.move("simulation_params_root_seed_42.tsv", os.path.join(output_dir,"simulation_params_root_seed_42.tsv"))
shutil.move("FR_boxplot_root_seed_42_all_reductions.png", os.path.join(output_dir,"FR_boxplot_root_seed_42_all_reductions.png"))

for file in glob.glob('simulation_results_root_seed_42*.tsv'):
    shutil.move(file, os.path.join(output_dir,file))
